In [ ]:
# default_exp preprocessing
%load_ext autoreload
%autoreload 2

# pre_processing


> API details.

In [ ]:
#hide
from nbdev.showdoc import *

![preprocessing](images/graphic8.PNG)

In [ ]:
#export
from fastai.vision.all import *
from fastai.medical.imaging import *
from torchvision.utils import save_image

### Mask & Save

In [ ]:
#export
def mask_and_save(df, source=None, show=None, window=dicom_windows.lungs, sigma:float=0.1, thresh:float=0.9, save=False, save_path=None):
    image_list = []
    for i in df.index:
        file_path = f"{source}/{df.iloc[i]['PatientID']}/{df.iloc[i]['InstanceNumber']}.dcm"
        file_name = df.iloc[i]['InstanceNumber']
        dcm = dcmread(file_path)
        wind = dcm.windowed(*window)
        mask = dcm.mask_from_blur(window, sigma=sigma, thresh=thresh, remove_max=False)
        bbs = mask2bbox(mask)
        lo,hi = bbs
        imh = wind[lo[0]:hi[0],lo[1]:hi[1]]
        if save is not False:
            save_image(imh, f'{save_path}/{file_name}.png')
        else:
            pass
        image_list.append(imh)
    if show is not None:
        show_images(image_list[:10], nrows=1)

In [ ]:
show_doc(mask_and_save)

<h4 id="mask_and_save" class="doc_header"><code>mask_and_save</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>mask_and_save</code>(**`df`**, **`source`**=*`None`*, **`show`**=*`None`*, **`window`**=*`(1500, -600)`*, **`sigma`**:`float`=*`0.1`*, **`thresh`**:`float`=*`0.9`*, **`save`**=*`False`*, **`save_path`**=*`None`*)



### Dicom metadata dict

Change to allow selection of dicom window width and level

In [ ]:
#export
@patch
def updated_dict(self:DcmDataset, windows=[dicom_windows.lungs]):
    pxdata = (0x7fe0,0x0010)
    vals = [self[o] for o in self.keys() if o != pxdata]
    its = [(v.keyword, v.value) for v in vals]
    res = dict(its)
    res['fname'] = self.filename
    
    stats = 'min', 'max', 'mean', 'std'
    pxs = self.pixel_array
    for f in stats: res['img_'+f] = getattr(pxs, f)()
    res['img_pct_window'] = self.pct_in_window(*windows)
    res['file_path'] = f'{self.PatientID}/{self.InstanceNumber}.dcm'
    return res

In [ ]:
#export
def _dcm2dict2(fn, windows, **kwargs): return fn.dcmread().updated_dict(windows, **kwargs)

In [ ]:
#export
@delegates(parallel)
def _from_dicoms2(cls, fns, n_workers=0, **kwargs):
    return pd.DataFrame(parallel(_dcm2dict2, fns, n_workers=n_workers, **kwargs))
pd.DataFrame.from_dicoms2 = classmethod(_from_dicoms2)

### dicom convert 3channel

In [ ]:
#export
def dicom_convert_3channel(fn:(Path,str), save_dir:(str), win1=dicom_windows.lungs, \
                           win2=dicom_windows.liver, win3=dicom_windows.brain):
    "Split a dicom image into 3 windows with each window per channel and saved as jpg"
    data = dcmread(fn)
    file_name = str(fn); name = file_name.split('\\')[-1].split('.')[0]
        
    chan_one = np.expand_dims(data.windowed(*win1), axis=2)
    chan_two = np.expand_dims(data.windowed(*win2), axis=2)
    chan_three = np.expand_dims(data.windowed(*(win3)), axis=2)
    image = np.concatenate([chan_one, chan_two, chan_three], axis=2)
    ten_image = TensorImage(image).permute(2,0,1)
    save_image(ten_image, f'{save_dir}/{name}.jpg')

In [ ]:
show_doc(dicom_convert_3channel)

<h4 id="dicom_convert_3channel" class="doc_header"><code>dicom_convert_3channel</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>dicom_convert_3channel</code>(**`fn`**:`Path'>, <class 'str'>)`, **`save_dir`**:`str`, **`win1`**=*`(1500, -600)`*, **`win2`**=*`(150, 30)`*, **`win3`**=*`(80, 40)`*)

Split a dicom image into 3 windows with each window per channel and saved as jpg

### Dicom Splitter

Open dicom splitter tutorial in Kaggle

[<img src="images/kaggle.PNG" width="80" align='left'/>](https://www.kaggle.com/avirdee/dicom-splitter-tutorial/)

In [ ]:
#export
def dicomsplit(valid_pct=0.2, seed=None, **kwargs):
    "Splits `items` between train/val with `valid_pct`"
    "and checks if identical patient IDs exist in both the train and valid sets"
    def _inner(o, **kwargs):
        train_list = []; valid_list = []
        if seed is not None: torch.manual_seed(seed)
        rand_idx = L(int(i) for i in torch.randperm(len(o)))
        cut = int(valid_pct * len(o))
        trn = rand_idx[cut:]; trn_p = o[rand_idx[cut:]]
        val = rand_idx[:cut]; val_p = o[rand_idx[:cut]]
        train_patient = []; train_images = []
        for i, tfile in enumerate(trn_p):
            file = dcmread(tfile)
            tpat = file.PatientID
            train_patient.append(tpat)
            file_array = dcmread(tfile).pixel_array
            train_images.append(file_array)
        val_patient = []; val_images = []
        for i, vfile in enumerate(val_p):
            file2 = dcmread(vfile)
            vpat = file2.PatientID
            val_patient.append(vpat)
            val_array = dcmread(vfile).pixel_array
            val_images.append(val_array)
    
        print(rand_idx)
        print(f'Train: {trn}, {train_patient}')
        show_images(train_images[:20])
        print(f'Val: {val}, {val_patient}')
        show_images(val_images[:20])
        is_duplicate = set(train_patient) & set(val_patient)
        print(f'Duplicate: {set(train_patient) & set(val_patient)}')
        new_list = []
        if bool(is_duplicate) is not False:
            print('duplicate exists')
            new_list = [elem for elem in train_patient if elem not in val_patient ]
            print(f'New List: {new_list}')
        else:
            print('duplicate does NOT exist')
            new_list = trn
        return new_list, val
    return _inner

In [ ]:
show_doc(dicomsplit)

<h4 id="dicomsplit" class="doc_header"><code>dicomsplit</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>dicomsplit</code>(**`valid_pct`**=*`0.2`*, **`seed`**=*`None`*, **\*\*`kwargs`**)

Splits `items` between train/val with `valid_pct`

In [ ]:
#export
def check_duplicate(items, seed=5):
    trn, val = dicomsplit(valid_pct=0.2, seed=seed)(items)
    return trn, val

In [ ]:
#export
def dicom_splitter(items, valid_pct=0.2, seed=77):
    trn, val = dicomsplit(valid_pct=valid_pct)(items)
    valid_idx = val
    def _inner(o):
        train_idx = np.setdiff1d(np.array(range_of(o)), np.array(valid_idx))
        print(f'train:{train_idx} val:{valid_idx}')
        return L(train_idx, use_list=True), L(valid_idx, use_list=True)
    return _inner

In [ ]:
show_doc(dicom_splitter)

<h4 id="dicom_splitter" class="doc_header"><code>dicom_splitter</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>dicom_splitter</code>(**`items`**, **`valid_pct`**=*`0.2`*, **`seed`**=*`77`*)



### Move files

Open move files tutorial in Kaggle

[<img src="images/kaggle.PNG" width="80" align='left'/>](https://www.kaggle.com/avirdee/pre-processing-tutorial/)

In [ ]:
#export
def move_files(df, source, save_path):
    "helper to move files"
    for i in df.index:
        #patient ID
        patid = str(df.PatientID[i])
        window = str(df.img_pct_window[i])
        
        #fname
        filename = str(df.fname[i]).split('/')[-1]
        img = filename.split('.')[0]
        print(f'ID: {patid} window: {window} instance: {img}')
        
        folder_path = save_path + patid
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)   
        img_file = Path(f'{source}/train/{patid}/{img}.dcm')
        shutil.copy(img_file, folder_path, follow_symlinks=True)

In [ ]:
show_doc(move_files)

<h4 id="move_files" class="doc_header"><code>move_files</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>move_files</code>(**`df`**, **`source`**, **`save_path`**)

helper to move files

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 02_explore.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_pipeline.ipynb.
Converted 05_train.ipynb.
Converted 06_examine.ipynb.
Converted 90_tutorial.ipynb.
Converted index.ipynb.
